# Auto Difference

* Dual number
    * definition
    * application --- calculate derivative of elementary function
    * dual number in Julia
* Auto-Differetiation (forward mode)

>>**Automatic differentiation** uses a sequence of elementary arithmetic operations and elementary functions (such as addition, multiplication, sine, cosine, and logarithm) to calculate the exact derivative of a function at any point in the function's domain. The process involves breaking down the function into smaller sub-functions, each of which has a known derivative, and then using the chain rule and product rule to combine these sub-functions to obtain the derivative of the original function.

There are at least three ways (symbolic, numerical and auto-differentiation) to calculate differentiation in computer. This Note provides intuitive explanation about forward mode Auto-Differentiation. Before showing how computer do differentiation computation by forward mode Auto Differentiation(AD), we will first introduce idea of dual number which help to get derivative of simple function and is closely related to forward mode AD. And next we back to algorithm of forward mode AD.
  
- automatic differentiation refers to computer algorithms
- AD is implemented by chain rules + dual number
- https://en.wikipedia.org/wiki/Automatic_differentiation

## Dual number
 

Dual numbers extend the real numbers and have the form $a + b\epsilon $:

- $a,b \in \mathbb{R}$ are real numbers;
- $\epsilon := (0, 1)$ is nilpotent of order two; that is, $\epsilon^n = 0$ for any integer $n \geq 2$;
    - "In mathematics, an element $x$ is called nilpotent if there exists some positive integer $n$ (i.e., *degree*, *order*) such that $x^n = 0$."
    - Intuitively, you might think of it as an infinitesimal number such that $\epsilon^2 = 0$ by the machine precision.
- $a$ is called the real part and $b$ is the dual part of the number.

Operations of dual number are similar to complex number and many of them look intuitive.

### addition and multiplication

\begin{align}
(a_1 + b_1\epsilon) + (a_2 + b_2\epsilon) & = (a_1 + a_2) + (b_1+b_2)\epsilon, \\
(a_1 + b_1\epsilon) * (a_2 + b_2\epsilon) & = a_1a_2 + (a_1b_2+a_2b_1)\epsilon + b_1b_2\epsilon^2\notag \\
                                          & = a_1a_2 + (a_1b_2+a_2b_1)\epsilon,\\
 c*(a_1+b_1\epsilon) & = ca_1+cb_1\epsilon, \\
\end{align} 
 

### division


\begin{align} 
\frac{a_1+b_1\epsilon}{a_2+b_2\epsilon} & = \frac{(a_1+b_1\epsilon)(a_2-b_2\epsilon)}{(a_2+b_2\epsilon)(a_2-b_2\epsilon)} = \frac{a_1a_2-a_1b_2\epsilon + b_1a_2\epsilon -b_1b_2\epsilon^2}{a_2^2+a_2b_2\epsilon-a_2b_2\epsilon-b_2^2\epsilon^2} \notag \\
            & = \frac{a_1a_2-a_1b_2\epsilon + b_1a_2\epsilon-0}{a_2^2-0} \notag \\
            & = \frac{a_1}{a_2} +\frac{b_1a_2-a_1b_2}{a_2^2}\epsilon.
\end{align}

[//]: # "Note, for the division, the final result could be intuitively interpreted as 
\begin{equation*}
\frac{f(x) + f(x)'\epsilon}{g(x) + g(x)'\epsilon} = \frac{f(x)}{g(x)} + \left(\frac{f(x)}{g(x)}\right)' \epsilon.
\end{equation*}"

## Add methods

Now Let's add the rules of dual numbers to Julia's methods.

In [3]:
import Base: +, *, -, ^, exp, log

struct DualNumber{T1, T2} <: Real   
    re::T1                # differnt types so (Float64, Int64), etc., is possible
    du::T2
end

+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.re + a, x.du)  
+(a::Number, x::DualNumber) = DualNumber(x.re + a, x.du) 

-(x::DualNumber, y::DualNumber) = DualNumber(x.re - y.re, x.du - y.du)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.re - a, x.du)  
-(a::Number, x::DualNumber) = DualNumber(a - x.re , -x.du)  

*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
*(x::DualNumber, a::Number) = DualNumber(a*x.re, a*x.du)
*(a::Number, x::DualNumber) = DualNumber(a*x.re, a*x.du)

# Where is the division rule? Your turn!

/ (generic function with 3 methods)

In [2]:
# Let's check the new rules on dual numbers.

y = DualNumber(2.0, 3)  # a dual number y = 2 + 3ϵ; note different types

@show typeof(y)
@show y.re      # check the real part
@show y.du      # check the dual part

   ## combinations of a dual number and a scalar

@show s1 = y + 2.5
@show typeof(s1)
@show s1.re
@show s1.du

println()

@show s2 = 2.5*y
@show typeof(s2)


  ## combinations of two dual numbers

z = DualNumber(0.5, 0.6)

println()

@show z
@show w1 = y+z
@show typeof(w1)

println()

@show w2 = y*z
# @show w3 = y/z

typeof(y) = DualNumber{Float64, Int64}
y.re = 2.0
y.du = 3
s1 = y + 2.5 = DualNumber{Float64, Int64}(4.5, 3)
typeof(s1) = DualNumber{Float64, Int64}
s1.re = 4.5
s1.du = 3

s2 = 2.5y = DualNumber{Float64, Float64}(5.0, 7.5)
typeof(s2) = DualNumber{Float64, Float64}

z = DualNumber{Float64, Float64}(0.5, 0.6)
w1 = y + z = DualNumber{Float64, Float64}(2.5, 3.6)
typeof(w1) = DualNumber{Float64, Float64}

w2 = y * z = DualNumber{Float64, Float64}(1.0, 2.7)


DualNumber{Float64, Float64}(1.0, 2.7)

 
### others

For other operations such as power rules and exponential function, we may use Taylor expansion to help us derive results. 

#### Taylor expansion: preliminary

Recall that we can expand a function $f(x)$ at the point of $x=a$ using the Taylor series:

\begin{align}
f(x) = f(a) + \frac{1}{1!}f'(a)(x-a) + \frac{1}{2!}f''(a)(x-a)^2 + \frac{1}{3!}f'''(a)(x-a)^3 + \ldots.
\end{align}

The equation says that we can use information about $f(a)$ (viz, the right-hand-side of the above equation) to approximate the unknown function $f(x)$ near $x=a$. So, if we now want to know the value of $f(x)$ evaluated at $x=c$, using the approximation equation we have

\begin{align}
f(c) = f(a) + f'(a)(c-a) + \frac{1}{2!}f''(a)(c-a)^2 + \frac{1}{3!}f'''(a)(c-a)^3 + \ldots.
\end{align}



Here comes the key part: Suppose now we want to use the approximation equation to get the value of $f(x)$ at the dual number of $x=a+b\epsilon$ (which means $x-a = b\epsilon$). The value would be:

\begin{align}
f(a+b\epsilon) & = f(a) + f'(a)b\epsilon + \frac{1}{2}f''(a)(b\epsilon)^2  + \ldots \\
\Longrightarrow \quad  f(a+b\epsilon)  & = f(a) + f'(a) b \epsilon.
\end{align}


Notice that the RHS of the equation has only two terms. It is because $\epsilon^n =0$ for $n\geq2$ and so the higher order terms are all equal to 0. Now look at the 2nd term on the RHS of the equation: There is an $f'(a)$, which is the derivative of $f(x)$ evaluated at $x=a$. This is the key result that helps us to get the derivative of $f(x)$ at a given value. We'll come back to this important point later. Before that, we will use the equation to derive operation rules for dual numbers.


#### apply to the power rule

Let $f(x) = x^n$ and so $f'(x) = nx^{n-1}$. For a dual number, it means $f(a+b\epsilon) = (a + b\epsilon)^n$.  Using the result of the Taylor expansion, we have

\begin{align}
   f(a+b\epsilon) & = f(a) + f'(a)b\epsilon = a^n + na^{n-1}b\epsilon;\\
  \Longrightarrow \quad (a+b\epsilon)^n & = a^n + na^{n-1}b\epsilon. 
\end{align}

Note that the first "=" is according to the Taylor expansion of $f(x)$, and the 2nd "=" follows from the definition of $f(x)=x^n$ and $f'(x)=nx^{n-1}$.

#### apply to the exponential function

Let $f(x) = e^x$ and so $f'(x) = e^x$. For a dual number, it means $f(a+b\epsilon) = e^{a+b\epsilon}$. Applying the Taylor expansion, we have

\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon = e^a + e^a b\epsilon; \\
\Longrightarrow \quad e^{a+b\epsilon} & = e^a + e^a b\epsilon.
\end{align}

Again, the first "=" is according to the Taylor expansion of $f(x)$, and the 2nd "=" follows from the definition of $f(x)$ and $f'(x)$.


#### how about the log() function? Your exercise!

In [3]:
# Let's add the power rule and the exponential method and see how they work

^(x::DualNumber, n::Union{Float64, Int64}) = DualNumber(x.re^n, n*x.re^(n-1)*x.du)  # cannot use n<:Real, since n is variable

exp(x::DualNumber) = DualNumber(exp(x.re), exp(x.re)*x.du)

y = DualNumber(2.0, 3.0)  # a dual number y = 2 + 3ϵ

@show u1 = exp(y)    # take exponential of the dual number; should be exp(2) + exp(2)*3*ϵ
@show u1.re
@show u1.du

println()

@show exp(2)    # check with the real part
@show exp(2)*3  # check with the dual part

@show typeof(-2)

@show y^(3)    # y^(-(1))

u1 = exp(y) = DualNumber{Float64, Float64}(7.38905609893065, 22.16716829679195)
u1.re = 7.38905609893065
u1.du = 22.16716829679195

exp(2) = 7.38905609893065
exp(2) * 3 = 22.16716829679195
typeof(-2) = Int64
y ^ 3 = DualNumber{Float64, Float64}(8.0, 36.0)


DualNumber{Float64, Float64}(8.0, 36.0)

## Automatic Differentiation: Combinations of Chain Rules and Dual Numbers

Recall that for a function $f(x)$, if $x$ is evaluated at a dual number $a+b\epsilon$, we have

\begin{align}
  f(a+b\epsilon) = f(a) + f'(a) b \epsilon.
\end{align}

Let $b=1$, then

\begin{align}
  f(a+\epsilon) = f(a) + f'(a)  \epsilon.
\end{align}

Notice that the dual part of the equation, $f'(a)$, is the derivative of $f(x)$ evaluated at $x=a$. 

>> **The result provides a method of obtaining the derivative of $f(x)$ at $x=a$: We simply augment $a$ to a dual number, evaluate the dual number in $f(x)$, and the dual part of the result is the derivative $f'(a)$.**

More specifically,

- Augment $x=a$ by a nilpotent $\epsilon$ and get the dual number $\tilde{x} = a + \epsilon$.

- Plug in $\tilde{x}$ to $f(x)$ and get $f(\tilde{x}) = f(a+\epsilon) = f(a) + f'(a)\epsilon$.

- The dual part of $f(\tilde{x})$, which is $f'(a)$, is the derivative of $f(x)$ evaluated at $x=a$.

What seems _**miracle**_ is that in the _**transform $->$ evaluation process**_, we do not conduct any kind of _**derivative**_ in the traditional sense. We simply evaluate the dual number at the function level, and we obtain the derivative _**for free**_!  But is it really a miracle? Does the derivative really come from nowhere? Surely not. The calculation has been done in our definition of dual number operations. To see this, let's try a simple example where we want to calculate the derivative of $f(y) = 3y^2 + 1$.

In [4]:
x0 = 2.0

f(y) = 3*y^2 + 1
df(y) = 6y       # true answer, derived by hand

x = DualNumber(x0, 1.0)
mydf = f(x)

@show mydf.du   # result from dual number
@show df(2.0)   # result from analytic equation 

mydf.du = 12.0
df(2.0) = 12.0


12.0

The example shows that the dual-number-based derivative is as good as that by hand calculation. We obtain the result by simply  plugging in $a+\epsilon$ into function and then read from the dual part of the result.

Why does it work? Where does the derivative come from? Well, this equation involves a power function, and the derivative of the power function is already there when we define the power rule of dual numbers.

So this example is straightforward. But let's consider a slightly complicated example which will motivate us to look at **"automatic differentiation"**.

Let $f(x) = e^{e^x}$ and we will use the dual number to calculate the function's derivative at $x=2$. 

In [5]:
x0 = 1  

f(x) = exp(exp(x))
df(x) = exp(exp(x))*exp(x)  # true answer, derived by hand

x_tilde = DualNumber(x0, 1.0) # dual number, with the real part being the target value

res = f(x_tilde)       # evaluate the function at the dual number

@show res.du           # should be the derivative of f(x) at x=x0.

@show df(x0)           # check with the symbolic equation

res.du = 41.193555674716116
df(x0) = 41.193555674716116


41.193555674716116

Again, the derivative based on dual numbers is easily obtained and is amazingly accurate, but where does the result come from? We didn't define the rule for the exponential-exponential function for dual numbers, right? 

Intuitively, the double exponential function is nothing but applying the the exponential function twice. Thus, the dual number is applied twice in the process and the result is obtained. 

_**Q: Who applied the exponential function twice? I certainly did not. I just click the button.**_


_**A: Computer did. To evaluate a function, a computer's algorithms would apply chain rules repeatedly on sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). Using dual numbers and applying chain rules, we get the automatic differentiation.**_

In [6]:
#=
import Base: convert, promote_rule
convert(::Type{DualNumber}, x::Real) = DualNumber(x, zero(x))
promote_rule(::Type{DualNumber}, ::Type{<:Number}) = DualNumber
=#

### Computational Graph

Auto Differentiation exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). By applying the chain rule repeatedly to these operations, we get differentiations on functions that may appear to be complicated. 

For example, let $f(x) = \frac{x^{1-r} - 1}{1-r}$ and find $f'(x)$ at $x=2$. Below is a computation graph of $f(x)$, decomposing $f(x)$ into several elementary operations. 

![title](auto_diff2.png)

|  Trace  |    Value ($v_i$)    |                         Derivative ($v_i'$)                        |                 dual number $$(v_i,v_i')$$                |
|:-------:|:-------------------:|:------------------------------------------------------------------:|:---------------------------------------------:|
| $$v_1$$ |     $$x^{1-r}$$     | $$v_1'=(1-r)x^{-r}$$                                               |      $$\left(2^{1-r},(1-r)2^{-r}\right)$$     |
| $$v_2$$ |      $$v_1-1$$      | $$v_2' = \frac{\partial v_2}{\partial v_1} v_1'=v_1'$$             |    $$\left(2^{1-r}-1,(1-r)2^{-r} \right)$$    |
| $$v_3$$ | $$\frac{v_2}{1-r}$$ | $$v_3' = \frac{\partial v_3}{\partial v_2} v_2'=\frac{v_2'}{1-r}$$ | $$\left(\frac{2^{1-r}-1}{1-r},2^{-r}\right)$$ |

The final result we wish to obtain is $\frac{d}{d x} v3$ which, using the chain rule, is $\frac{1}{1-r}\frac{\partial}{\partial x}v_2$. The $\frac{\partial}{\partial x}v_2$ could also be calculated using the chain rule by $\frac{\partial}{\partial x}v_1$. We can do the similar thing to $\frac{\partial}{\partial x}v_1$.

The insight here is that in order to get $\frac{df}{dx}$, we will use all of the values and derivatives of each nodes, and the calculation on each node involves only elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). We have defined how dual numbers should behave in these operations and functions, and so we could apply dual numbers to $f(x)$, which will then be carried over through the nodes and produce $(v_i, v_i')$ at each node. The final answer is the dual part of the final evaluation.

The above sequence of operations can be carried out either forward (_**forward mode**_) or backward (_**reverse mode**_). We show how they are done using the example above. Recall we have $y=f(x)=v_3$.

### Forward mode
$$\begin{align}
    \frac{\partial y}{\partial x} &= \frac{\partial v_3}{\partial v_2}\frac{\partial v_2}{\partial x} = \frac{\partial v_3}{\partial v_2}\left(\frac{\partial v_2}{\partial v_1}\frac{\partial v_1}{\partial x}\right) \\
    \notag\\
    &= \frac{\partial v_3}{\partial v_2}\left(\frac{\partial v_2}{\partial v_1} (1-r)x^{-r}\right) \\
    \notag\\
    &= \frac{\partial v_3}{\partial v_2}(1-r)x^{-r} \\
    \notag\\
    &= \frac{1}{1-r}(1-r)x^{-r} = x^{-r}
\end{align}$$ 

Note that the order of calculation is: $\frac{\partial v_1}{\partial x}$ -> $\frac{\partial v_2}{\partial v_1}$ ->  $\frac{\partial v_3}{\partial v_2}$. Thus the "forward mode".

### Reverse mode
$$\begin{align}
    \frac{\partial y}{\partial x} &= \frac{\partial v_3}{\partial v_1}\frac{\partial v_1}{\partial x} = \left(\frac{\partial v_3}{\partial v_2}\frac{\partial v_2}{\partial v_1} \right)\frac{\partial v_1}{\partial x} \\
    \notag \\
    &= \left(\frac{1}{1-r}\frac{\partial v_2}{\partial v_1}\right)\frac{\partial v_1}{\partial x} \\
    \notag \\
    &= \left(\frac{1}{1-r}\right)\frac{\partial v_1}{\partial x} \\
    \notag \\
    &= \left(\frac{1}{1-r}\right)(1-r)x^{-r} = x^{-r}
\end{align}$$

Note that the order of calculation is: $\frac{\partial v_3}{\partial v_2}$ -> $\frac{\partial v_2}{\partial v_1}$ ->  $\frac{\partial v_1}{\partial x}$. Thus the "reverse mode".

Remark: When implemented in computer algorithms, the reverse mode tend to be more efficient (less calculation) but it is harder to code and requires more storage.

### Autodiff for 2nd Derivatives

Extending the above procedures to 2nd derivatives is not straightforward. One way to do it is to use specialized algorithms that, in a sense, takes another round of froward and reverse sweep on the computational graph. 

Another way is to use _**hyper dual numbers**_ which has one real part and three non-real parts, as follows:

\begin{equation}
  c = a + b_1 \epsilon_1 + b_2 \epsilon_2 + b_3 \epsilon_1 \epsilon_2,
\end{equation}
where
\begin{align}
   \epsilon_1^2 = \epsilon_2^2 =0, \\
   \epsilon_1\epsilon_2 = \epsilon_2 \epsilon_1 \neq 0.
\end{align}

Applying the hyper dual numbers to calculate second derivatives follows more or less similar procedures as we have discussed above on 1st derivatives, but the details are beyond the scope of this lecture.

In [7]:
# Pkg.build("PyCall")

In [8]:
# using Pkg; Pkg.add("SymPy")
using SymPy

In [9]:
# Define each elementary operation used and its derivative. This part is only for illustration.
function v1(x,r)
    return x^(1-r)
end

function v2(x,r)
    return x-1
end

function v3(x,r)
    return x* (1-r)^(-1.0)
end

function dv1(x,r)
    return (1-r) * x^(-r)
end

function dv2(x,r)
    return dv1(x,r)    #？
end

function dv3(x,r)
    return dv2(x,r) * (1-r)^(-1.0)
end

# Symbolic representation
x, r = symbols("x,r")
display("Elementary operations of each node " )
v1(x,r) |> display
v2(v1(x,r),r) |> display
v3(v2(v1(x,r),r),r) |> display

display("Derivatives of each node " )
dv1(x,r) |> display
dv2(x,r) |> display
dv3(x,r) |> display

"Elementary operations of each node "

 1 - r
x     

 1 - r    
x      - 1

       -1.0 / 1 - r    \
(1 - r)    *\x      - 1/

"Derivatives of each node "

 -r        
x  *(1 - r)

 -r        
x  *(1 - r)

 -r
x  

In [10]:
# define f(x) = (x^{1-r}-1)/(1-r)

function f_AD_Sym(x,r) # auto diff's 的內部操作; no use of dual numbers
    f = x^(1-r)
        df = (1-r) * x^(-r)
    f = f-1
        df = df
    f = f * (1-r)^(-1.0)
        df = df * (1-r)^(-1.0)
    return df
end



function f_AD_DualNum(x,r) # we've defined DualNumber for +,-,*,^, so all the operation should base on these
    f = x^(1-r)  
    f = f-1
    f = f * (1-r)^(-1.0)
    return f
end



f_AD_DualNum (generic function with 1 method)

In [11]:
display("The symbolic form of the derivative of f(x) ")
f_AD_Sym(x,r) |> display

"The symbolic form of the derivative of f(x) "

 -r
x  

In [12]:
# The derivative of f(x) (with r=2.0) at x=2 using symbolic form
f_AD_Sym(2, 2.0)

0.25

In [13]:
# The derivative of f(x) (with r=2.0) at x=2 using Dual Numbers

f_AD_DualNum(DualNumber(2,1), 2.0)

DualNumber{Float64, Float64}(0.5, 0.25)

## reference:

https://harvard-iacs.github.io/2019-CS207/lectures/lecture10/notebook/


https://en.wikipedia.org/wiki/Automatic_differentiation

https://github.com/MikeInnes/diff-zoo

https://julia.quantecon.org/more_julia/optimization_solver_packages.html

https://arxiv.org/pdf/1502.05767.pdf

https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

https://towardsdatascience.com/build-your-own-automatic-differentiation-program-6ecd585eec2a

